In [31]:
import pandas as pd

Read all data from csv file.

In [48]:
word_association_men = pd.read_csv('EDA_with_Pandas.csv')
word_association_women = pd.read_csv('women_EDA.csv')

In [49]:
cols = ['Загальна кількість реакцій', 'Кількість найчастіших реакцій']
word_association_men.loc[:,cols] = word_association_men.loc[:,cols].ffill()
word_association_men.head()

,Стимул,Реакція,Кількість реакцій,Загальна кількість реакцій,Кількість найчастіших реакцій,Тип асоціації
0,зауваження,вчитель,4,101.0,16.0,слово контексту
1,зауваження,вчителя,4,101.0,16.0,слово контексту
2,зауваження,порада,4,101.0,16.0,синонім
3,зауваження,школа,4,101.0,16.0,слово контексту
4,зауважити,помилку,11,100.0,30.0,слово контексту


In [ ]:
type_association = word_association_men['Тип асоціації'].value_counts()
type_association

Тип асоціації
слово контексту    780
синонім            219
антонім             33
Name: count, dtype: int64

In [59]:
the_most_frequent_reactions_of_men = word_association_men.groupby(['Реакція', 'Стимул'], as_index=False)['Кількість реакцій']\
                                    .sum().sort_values(by='Кількість реакцій', ascending=False).set_index('Стимул')
the_most_frequent_reactions_of_men.head()

,Реакція,Кількість реакцій
Стимул,,
неприємний,запах,43
калина,червона,41
пізнати,себе,41
питання,відповідь,40
країна,Україна,39


In [66]:
the_most_frequent_reactions_of_women = word_association_women.groupby(['Реакція', 'Стимул'], as_index=False)['Кількість реакцій']\
                                    .sum().sort_values(by='Кількість реакцій', ascending=False).set_index('Стимул')
the_most_frequent_reactions_of_women.head()

,Реакція,Кількість реакцій
Стимул,,
завтрашній,день,85
сьогоднішній,день,74
заробити,гроші,70
заробляти,гроші,69
вірний,друг,62
